In [1]:
import ipywebgl
import numpy as np

In [2]:
w = ipywebgl.GLViewer()
w.clear_color(.8, .8, .8 ,1)
w.clear()
w.enable(cull_face=True)
w.execute_commands(execute_once=True)

In [3]:
fb = w.create_framebuffer()
w.execute_commands(execute_once=True)

In [4]:
shadow_prog = w.create_program_ext(
"""#version 300 es

uniform mat4 u_lightProjection;
uniform mat4 u_world;
in vec3 in_vert;

void main() {
    gl_Position = u_lightProjection * u_world * vec4(in_vert, 1.0);
}
"""
,
"""#version 300 es
precision highp float;
out vec4 f_color;
void main() {
    f_color = vec4(1, 0.1, 0.1, 1.0);
}
""")

In [6]:
cube_vbo = w.create_buffer_ext(
    src_data=np.array(
      [[ 0.  ,  0.  ,  1.  ],
       [-0.72, -0.53,  0.45],
       [ 0.28, -0.85,  0.45],
       [ 0.89,  0.  ,  0.45],
       [ 0.28,  0.85,  0.45],
       [-0.72,  0.53,  0.45],
       [-0.89, -0.  , -0.45],
       [-0.28, -0.85, -0.45],
       [ 0.72, -0.53, -0.45],
       [ 0.72,  0.53, -0.45],
       [-0.28,  0.85, -0.45],
       [-0.  ,  0.  , -1.  ],
       [ 0.16, -0.5 ,  0.85],
       [-0.43, -0.31,  0.85],
       [-0.26, -0.81,  0.53],
       [ 0.53, -0.  ,  0.85],
       [ 0.69, -0.5 ,  0.53],
       [ 0.16,  0.5 ,  0.85],
       [ 0.69,  0.5 ,  0.53],
       [-0.43,  0.31,  0.85],
       [-0.26,  0.81,  0.53],
       [-0.85, -0.  ,  0.53],
       [-0.59, -0.81, -0.  ],
       [-0.  , -1.  , -0.  ],
       [ 0.59, -0.81,  0.  ],
       [ 0.95, -0.31, -0.  ],
       [ 0.95,  0.31, -0.  ],
       [ 0.59,  0.81, -0.  ],
       [-0.  ,  1.  , -0.  ],
       [-0.59,  0.81, -0.  ],
       [-0.95,  0.31,  0.  ],
       [-0.95, -0.31,  0.  ],
       [-0.69, -0.5 , -0.53],
       [ 0.26, -0.81, -0.53],
       [ 0.85,  0.  , -0.53],
       [ 0.26,  0.81, -0.53],
       [-0.69,  0.5 , -0.53],
       [-0.53, -0.  , -0.85],
       [-0.16, -0.5 , -0.85],
       [ 0.43, -0.31, -0.85],
       [ 0.43,  0.31, -0.85],
       [-0.16,  0.5 , -0.85]], dtype=np.float32).flatten()
)
indices = np.array(
        [[0, 13, 12], [12, 14, 2], [12, 13, 14], [13, 1, 14], [0, 12, 15], [15, 16, 3], [15, 12, 16], [12, 2, 16],
          [0, 15, 17], [17, 18, 4], [17, 15, 18], [15, 3, 18], [0, 17, 19], [19, 20, 5], [19, 17, 20], [17, 4, 20],
          [0, 19, 13], [13, 21, 1], [13, 19, 21], [19, 5, 21], [1, 22, 14], [14, 23, 2], [14, 22, 23], [22, 7, 23],
          [2, 24, 16], [16, 25, 3], [16, 24, 25], [24, 8, 25], [3, 26, 18], [18, 27, 4], [18, 26, 27], [26, 9, 27],
          [4, 28, 20], [20, 29, 5], [20, 28, 29], [28, 10, 29], [5, 30, 21], [21, 31, 1], [21, 30, 31], [30, 6, 31],
          [1, 31, 22], [22, 32, 7], [22, 31, 32], [31, 6, 32], [2, 23, 24], [24, 33, 8], [24, 23, 33], [23, 7, 33],
          [3, 25, 26], [26, 34, 9], [26, 25, 34], [25, 8, 34], [4, 27, 28], [28, 35, 10], [28, 27, 35], [27, 9, 35],
          [5, 29, 30], [30, 36, 6], [30, 29, 36], [29, 10, 36], [6, 37, 32], [32, 38, 7], [32, 37, 38], [37, 11, 38],
          [7, 38, 33], [33, 39, 8], [33, 38, 39], [38, 11, 39], [8, 39, 34], [34, 40, 9], [34, 39, 40], [39, 11, 40],
          [9, 40, 35], [35, 41, 10], [35, 40, 41], [40, 11, 41], [10, 41, 36], [36, 37, 6], [36, 41, 37], [41, 11, 37]],
        dtype=np.uint8).flatten()

cube_vao = w.create_vertex_array_ext(
    shadow_prog,
    [
        (cube_vbo, '3f32', 'in_vert'),
        
    ],
    indices
)

# light matrix on top looking down
light_matrix = np.eye(4)
light_matrix[:3, 3] = np.array([0,200,0])
light_matrix[:3, 2] = np.array([0,1,0])
light_matrix[:3, 1] = np.array([0,0,-1])
light_matrix = np.linalg.inv(light_matrix)

def ortho(width, height, near, far):
    A = 1. / width
    B = 1. / height
    C = -(far + near) / (far - near)
    D = -2. / (far - near)
    return np.array([
        [A, 0, 0, 0],
        [0, B, 0, 0],
        [0, 0, D, C],
        [0, 0, 0, 1]
    ], dtype=np.float32)

def m4ProjectionMatrix(fov_y, aspect_ratio, near, far):
    ymax = near * np.tan(fov_y * np.pi / 360.0)
    xmax = ymax * aspect_ratio

    return frustrum(-xmax, xmax, -ymax, ymax, near, far)


def frustrum(left, right, bottom, top, near, far):
    A = (right + left) / (right - left)
    B = (top + bottom) / (top - bottom)
    C = -(far + near) / (far - near)
    D = -2. * far * near / (far - near)
    E = 2. * near / (right - left)
    F = 2. * near / (top - bottom)

    return np.array([
        [E, 0, A, 0],
        [0, F, B, 0],
        [0, 0, C, D],
        [0, 0, -1, 0]
    ], dtype=np.float32)

light_projection = np.dot(ortho(4,4, 1.0, 5000.0), light_matrix)

sphere1 = np.eye(4)
sphere1[:3, 3] = np.array([0,1,0])
sphere2 = np.eye(4)
sphere2[:3, 3] = np.array([2,1,0])

w.clear()
w.use_program(shadow_prog)
w.uniform_matrix('u_lightProjection', light_projection.T)
w.bind_vertex_array(cube_vao)

w.uniform_matrix('u_world', sphere1.T)
w.draw_elements('TRIANGLES', indices.shape[0], 'UNSIGNED_BYTE', 0)
w.uniform_matrix('u_world', sphere2.T)
w.draw_elements('TRIANGLES', indices.shape[0], 'UNSIGNED_BYTE', 0)
# render in loop if needed
w.execute_commands()
w

GLViewer(camera_pos=[0, 50, 200])